In [ ]:
import os
from pyo import *

################
# SYNTHÉTISEUR #
################

def simple_synth(nom_fichier, params):
    """
    Génère un fichier audio basé sur les paramètres fournis.
    
    params : dictionnaire contenant la configuration du synthé.
    """
    
    s = Server(audio='offline')
    s.boot()
    
    duree_totale = params['duree'] + params['env_release']
    s.recordOptions(dur=duree_totale, filename=nom_fichier, fileformat=0, sampletype=0)

    # - OSCILLATEUR
    # LFO est utilisé ici comme oscillateur audio car il permet de changer 
    # la forme d'onde facilement via un numéro (0=Sine, 1=Rect, 2=Tri, 3=Saw, etc.)
    # type 3 = Saw (Dents de scie), type 1 = Pulse/Square
    osc = LFO(freq=params['osc_freq'], sharp=1, type=params['osc_type'], mul=params['osc_vol'])

    # - NOISE
    # Bruit blanc mélangé au signal
    noise = Noise(mul=params['noise_vol'])

    # Mixage Oscillateur + Bruit
    source_mix = osc + noise

    # - FILTRE
    # filtre multimode
    # type 0 = Lowpass, 1 = Highpass, 2 = Bandpass
    filt = Biquadx(source_mix, freq=params['filter_cutoff'], q=params['filter_res'], type=params['filter_type'])

    # - ENVELOPPE ADSR
    env = Adsr(attack=params['env_attack'], 
               decay=params['env_decay'], 
               sustain=params['env_sustain'], 
               release=params['env_release'], 
               dur=params['duree'],
               mul=params['master_vol'])
    
    # Application de l'enveloppe sur le signal filtré
    signal = filt * env
    panner = Pan(signal, outs=2, pan=0.5) 
    panner.out()

    # - RENDU
    env.play() # Déclenche l'enveloppe
    s.start()  # Lance le calcul audio
    
    # Nettoyage
    s.shutdown()

In [ ]:
##################
# EXEMPLE PRESET #
##################

parametres_synthe = {
    'duree': 1.0,           

    'osc_freq': 210.0,      # Fréquence (Hz)
    'osc_type': 0,          # 0=Saw Up, 1=Saw Down, 2=Square, 3=Triangle, 4=Pulse, 5=Bipolar Pulse, 7=Modulated Sine
    'osc_vol': 0.6,         # Volume de l'osc (0.0 à 1.0)
    
    'noise_vol': 0.1,       # Quantité de bruit (0.0 pour un son pur)
    
    'filter_type': 2,       # 0=LowPass, 1=HighPass, 2=BandPass
    'filter_cutoff': 800,   # Fréquence de coupure (Hz)
    'filter_res': 1.5,      # Résonance
    
    'env_attack': 0.05,     # Attaque (sec)
    'env_decay': 0.2,       # Decay (sec)
    'env_sustain': 0.7,     # Sustain (niveau 0.0 à 1.0)
    'env_release': 0.5,     # Release (sec)
    
    'master_vol': 0.8       # Volume (0.0 à 1.0)
}

# Génération
nom_sortie = "example.wav"
filepath = os.path.join("wavs", nom_sortie)
simple_synth(filepath, parametres_synthe)
print("example.wav enregistré dans le dossier 'wavs'")

In [3]:
#################
# DISTRIBUTIONS #
#################

def random_log(tuple_min_max):
    """
    Retourne une valeur aléatoire distribuée logarithmiquement.
    """
    log_min = math.log(tuple_min_max[0])
    log_max = math.log(tuple_min_max[1])
    
    log_val = random.uniform(log_min, log_max)
    
    return math.exp(log_val)

def random_pow(tuple_min_max, curve=3):
    """
    Retourne une valeur aléatoire distribuée exponentiellement.
    """
    raw = random.random() 
    valeur_courbe = raw ** curve
    
    return valeur_courbe * tuple_min_max[1]

def random_cutoff(osc_freq, filter_type):
    """
    Calcule un cutoff cohérent pour éviter le silence.
    filter_type : 0 = LowPass, 1 = HighPass, 2 = BandPass
    """

    min_global = CONFIGS['filter_cutoff'][0]
    max_global = CONFIGS['filter_cutoff'][1]
    
    # - LOW PASS (Type 0)
    # On garde les graves, donc on coupe AU-DESSUS de la note
    if filter_type == 0:
        low_bound = max(min_global, osc_freq) # Au minimum la note elle-même
        high_bound = max_global

    # - HIGH PASS (Type 1)
    # On garde les aigus, donc on coupe EN-DESSOUS de la note
    # (ou légèrement au-dessus si on veut un effet "téléphone" sans basse)
    elif filter_type == 1:
        low_bound = min_global
        # On permet d'aller un peu au-dessus de la fréquence fondamentale (ex: x1.5)
        # pour ne garder que les harmoniques (son très fin), mais pas trop loin sinon silence.
        high_bound = min(max_global, osc_freq * 1.5) 
        
        # Sécurité si la note est très grave
        if high_bound <= low_bound:
            return low_bound

    # - BAND PASS (Type 2)
    # On garde juste une bande autour de la note
    else:
        # On centre la recherche autour de la fréquence de la note
        low_bound = max(min_global, osc_freq * 0.5)
        high_bound = min(max_global, osc_freq * 2.0)

    # --- TIRAGE LOGARITHMIQUE ---
    # (Indispensable pour que la distribution soit naturelle à l'oreille)
    log_min = math.log(low_bound)
    log_max = math.log(high_bound)
    
    return math.exp(random.uniform(log_min, log_max))

In [ ]:
#######################
# DOSSIER & NETTOYAGE #
#######################

wav_dir = "wavs"
log_dir = "logs"

os.makedirs(wav_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

def clean_folder(chemin_dossier):
    """
    Supprime tous les fichiers présents dans le dossier indiqué.
    """
    # Récupère la liste de tous les fichiers dans le dossier
    fichiers = glob.glob(os.path.join(chemin_dossier, "*"))
    
    if not fichiers:
        print(f"Le dossier '{chemin_dossier}' est déjà vide.")
        return

    print(f"Nettoyage de '{chemin_dossier}' ({len(fichiers)} fichiers supprimés)...")
    for f in fichiers:
        try:
            os.remove(f) # Supprime le fichier
        except Exception as e:
            print(f"Erreur lors de la suppression de {f} : {e}")

clean_folder(wav_dir)
clean_folder(log_dir)

In [ ]:
import csv
import glob
import random
import numpy as np

######################
# GENERATION DATASET #
######################

csv_path = os.path.join(log_dir, "dataset_labels.csv")

CONFIGS = {
    'duree': (0.5, 4.0),

    'osc_freq': (40.0, 4000.0),
    'osc_type': [0, 1, 2, 3, 4, 5, 7],
    'osc_vol': (0.0, 1.0),

    'noise_vol': (0.0, 0.4), # Max 40% de bruit

    'filter_type' : [0, 1, 2],
    'filter_cutoff': (50.0, 20000.0),
    'filter_res': (0.0, 1.5),

    'env_attack': (0.001, 1.0),
    'env_decay': (0.01, 1.0),
    'env_sustain': (0.0, 1.0),
    'env_release': (0.01, 2.0),

    'master_vol': (0.8, 1.0)
}

NB_SAMPLES = 10000

with open(csv_path, mode='w', newline='') as f:
    writer = csv.writer(f)
    headers = ["filename", "duree", "osc_type", "osc_freq", "osc_vol", "noise_vol", "filter_type", 
               "filter_cutoff", "filter_res", "env_attack", "env_decay", "env_sustain", "env_release", 
               "master_vol"]
    writer.writerow(headers)

    for i in range(NB_SAMPLES):
        filename = f"sound_{i:05d}.wav" # ex: sound_XXXXX.wav
        filepath = os.path.join(wav_dir, filename)

        p = {}
        p['duree'] = random.uniform(*CONFIGS['duree'])

        p['osc_type'] = random.choice(CONFIGS['osc_type'])
        p['osc_freq'] = random_log(CONFIGS['osc_freq'])

        no_volume = True
        while no_volume:
            p['osc_vol'] = random.uniform(*CONFIGS['osc_vol'])
            p['noise_vol'] = random_pow(CONFIGS['noise_vol'], 2)
            no_volume = p['osc_vol'] == 0.0 and p['noise_vol'] == 0.0
        
        p['filter_type'] = random.choice(CONFIGS['filter_type'])
        p['filter_cutoff'] = random_cutoff(p['osc_freq'], p['filter_type'])
        p['filter_res'] = random.uniform(*CONFIGS['filter_res'])

        p['env_attack'] = random.uniform(*CONFIGS['env_attack'])
        p['env_decay'] = random.uniform(*CONFIGS['env_decay'])
        p['env_sustain'] = random.uniform(*CONFIGS['env_sustain'])
        p['env_release'] = random.uniform(*CONFIGS['env_release'])
        
        p['master_vol'] = random.uniform(*CONFIGS['master_vol'])

        simple_synth(filepath, p)
        row = [filename, p['duree'], p['osc_type'], p['osc_freq'], p['osc_vol'], p['noise_vol'], 
               p['filter_type'], p['filter_cutoff'], p['filter_res'], 
               p['env_attack'], p['env_decay'], p['env_sustain'], p['env_release'], p['master_vol']]
        writer.writerow(row)

print("Terminé ! Dataset prêt")

In [ ]:
import pandas as pd

##############
# VALIDATION #
##############

CSV_FILE = "logs/dataset_labels.csv"

def print_header(title):
    print("\n" + "="*60)
    print(f" {title.upper()}")
    print("="*60)

def analyze_dataset(csv_path):
    if not os.path.exists(csv_path):
        print(f"Erreur : Le fichier '{csv_path}' est introuvable.")
        return

    df = pd.read_csv(csv_path)
    
    # 1. VUE D'ENSEMBLE
    print_header("1. Vue d'ensemble")
    print(f"Nombre total d'échantillons : {len(df)}")
    print(f"Colonnes disponibles : {', '.join(df.columns)}")
    
    # Vérification des nulls
    null_counts = df.isnull().sum().sum()
    if null_counts == 0:
        print("Aucune valeur manquante (Null) détectée.")
    else:
        print(f"ATTENTION : {null_counts} valeurs manquantes détectées")

    # 2. DISTRIBUTION DES CATÉGORIES
    print_header("2. Distribution des Catégories")
    
    # Oscillateurs
    osc_counts = df['osc_type'].value_counts().sort_index()
    print("\n--- Types d'Oscillateurs (0-7) ---")
    print("0=Saw Up, 1=Saw Down, 2=Square, 3=Triangle, 4=Pulse, 5=Bipolar Pulse, 7=Modulated Sine\n")
    print(osc_counts.to_string())

    # Filtres
    filter_counts = df['filter_type'].value_counts().sort_index()
    print("\n--- Types de Filtres (0=LP, 1=HP, 2=BP) ---")
    print(filter_counts.to_string())

    # 3. ANALYSE STATISTIQUE (Audio)
    print_header("3. Analyse Audio & Physique")
    
    stats = df[['osc_freq', 'filter_cutoff', 'noise_vol', 'filter_res']].describe().round(2)
    print(stats.loc[['mean', 'min', '50%', 'max']])
    
    print("\n--- Analyse de la distribution des Fréquences ---")
    median_freq = df['osc_freq'].median()
    mean_freq = df['osc_freq'].mean()
    print(f"Médiane : {median_freq:.2f} Hz | Moyenne : {mean_freq:.2f} Hz")
    if median_freq < mean_freq:
        print("Distribution Logarithmique OK (Médiane < Moyenne).")
        print("   Il y a une bonne concentration de basses/médiums.")
    else:
        print("Attention : La distribution semble linéaire (trop d'aigus).")

    # 4. SANITY CHECK (Détection d'erreurs physiques)
    print_header("4. Sanity Check (Détection de Silence)")
    
    # Check LowPass (Silence si Cutoff << Freq)
    # Seuil de tolérance : Cutoff < 0.9 * Freq (On accepte un peu de filtrage de la fondamentale)
    bad_lp = df[(df['filter_type'] == 0) & (df['filter_cutoff'] < df['osc_freq'] * 0.9)]
    
    # Check HighPass (Son "maigre" si Cutoff > Freq)
    # Seuil : Cutoff > 1.1 * Freq
    thin_hp = df[(df['filter_type'] == 1) & (df['filter_cutoff'] > df['osc_freq'] * 1.1)]

    if len(bad_lp) == 0:
        print("LowPass : Aucun risque de silence détecté.")
    else:
        print(f"LowPass : {len(bad_lp)} fichiers risquent d'être étouffés (Cutoff < Freq).")
        print(bad_lp[['filename', 'osc_freq', 'filter_cutoff']].head())

    if len(thin_hp) == 0:
        print("HighPass : Aucun filtrage excessif de la fondamentale.")
    else:
        print(f"HighPass : {len(thin_hp)} fichiers ont la fondamentale coupée (Son fin/maigre).")
        print(f"   (C'est acceptable tant que ce n'est pas 100% du dataset)")    

if __name__ == "__main__":
    analyze_dataset(CSV_FILE)
print("\n")